In [82]:
import os

dir_project = os.path.join(os.getcwd())
dir_data = os.path.join(dir_project, '..', 'nasa_lights')

#Set up relevant folder paths.
dir_nl = os.path.join(dir_project)
dir_nl_annual = os.path.join(dir_nl, 'annual')
dir_nl_monthly = os.path.join(dir_nl, 'monthly')
dir_nl_temp = os.path.join(dir_nl, 'temp') #Folder to store tiff images, may be deleted after calculating NLI

# csv_nl_annual = os.path.join(dir_data, 'nl', 'nl_workshop_sample_annual.csv')
# csv_nl_monthly = os.path.join(dir_data, 'nl', 'nl_workshop_sample_monthly.csv')

In [83]:
#Construct list of all folders containing raw hd5 images to convert.
nl_filepaths = [os.path.join(dir_data, 'monthly')]
nl_filepaths

['C:\\Users\\zfilk\\Documents\\Python Scripts\\sentinel\\..\\nasa_lights\\monthly']

In [84]:
#Function to convert HDFs to Tiffs
def hdf_to_tiff(path_in, path_out, sublayer):
    if path_in == path_out:
        warnings.warn('Input path is the same as output path. Cannot overwrite file.')
        return
        
    hdflayer = gdal.Open(path_in, gdal.GA_ReadOnly)
    if not os.path.exists(os.path.dirname(path_out)): os.makedirs(os.path.dirname(path_out))
    
    subhdflayer = hdflayer.GetSubDatasets()[sublayer]
    rlayer = gdal.Open(subhdflayer[0], gdal.GA_ReadOnly)
    
    HorizontalTileNumber = int(rlayer.GetMetadata_Dict()['HorizontalTileNumber'])
    VerticalTileNumber = int(rlayer.GetMetadata_Dict()['VerticalTileNumber'])     
    WestBoundCoord = (10*HorizontalTileNumber) - 180
    NorthBoundCoord = 90-(10*VerticalTileNumber)
    EastBoundCoord = WestBoundCoord + 10
    SouthBoundCoord = NorthBoundCoord - 10        
    EPSG = "-a_srs EPSG:4326" #WGS84
    translateOptionText = EPSG+" -a_ullr " + str(WestBoundCoord) + " " + str(NorthBoundCoord) + " " + str(EastBoundCoord) + " " + str(SouthBoundCoord) + ' -a_nodata 65535'
    translateoptions = gdal.TranslateOptions(gdal.ParseCommandLine(translateOptionText))
    gdal.Translate(path_out, rlayer, options=translateoptions)

In [85]:
#Use the snow-free and snow-covered NL sublayers
nl_sublayers = [0, 4]

#Convert specified sublayers of annual NL raw hd5 rasters into tiff files, merge mosaics.
for nl_filepath in nl_filepaths:
    hd5s = [os.path.join(nl_filepath, file) for file in os.listdir(nl_filepath) if os.path.join(nl_filepath, file).endswith('.h5')]
    for nl_sublayer in nl_sublayers:
        lst_tiffs = []
        for hd5 in hd5s:
            tiff = os.path.join(dir_nl_temp, os.path.splitext(os.path.basename(hd5))[0] + f'_{nl_sublayer}' + '.tif')
            hdf_to_tiff(path_in=hd5, path_out=tiff, sublayer=nl_sublayer)
            lst_tiffs.append(tiff)
        
        #Merge the mosaic tiff files
        if dir_nl_annual in nl_filepath:
            resolution = 'annual'
            year = nl_filepath.split(dir_nl)[1].split('/')[-1]
            month = None
            merged_tiff_file = os.path.join(dir_nl_temp, 'merged', 'annual', year, f'{year}_{nl_sublayer}.tif')
        elif dir_nl_monthly in nl_filepath:
            resolution = 'monthly'
            year =  nl_filepath.split(dir_nl)[1].split('/')[-2]
            month = nl_filepath.split(dir_nl)[1].split('/')[-1]
            merged_tiff_file = os.path.join(dir_nl_temp, 'merged', 'monthly', year, month, f'{year}_{month}_{nl_sublayer}.tif')
        
        if not os.path.exists(os.path.dirname(merged_tiff_file)):
            os.makedirs(os.path.dirname(merged_tiff_file))
            
        merged_raster, output_transform = merge(lst_tiffs)
        with rasterio.open(lst_tiffs[0]) as src:
            out_meta = src.meta.copy()
        out_meta.update({'driver' : 'GTiff', 'height' : merged_raster.shape[1], 'width' : merged_raster.shape[2], 'transform' : output_transform})
        with rasterio.open(merged_tiff_file, 'w', **out_meta) as out_raster:
            out_raster.write(merged_raster) 
        
        print(f'Built and saved merged tiff for year:{year}, month:{month}, layer:{nl_sublayer}.')

NameError: name 'gdal' is not defined